In [4]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

# import wandb

In [8]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x'


# cfg.model.roi_head.bbox_head.num_classes = 11


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [ ]:
# import os
# from visualization import push_image

# wandb_name = f"{cfg.model['Dongju-object_detection']}_v0"                 # wandb run name, 원하는 이름으로 변경


# cfg.log_config.hooks[1].init_kwargs.name = wandb_name  # loss, acc 기록과 시각화 기록이 다른 Run에 저장되어, 이름을 통일해 두기 위함.

# checkpoint_file = 'best_bbox_mAP.pth'
# checkpoint_path = os.path.join(cfg.work_dir, checkpoint_file)

In [9]:
model = build_detector(cfg.model)

2021-05-19 13:56:30,071 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-19 13:56:30,072 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-19 13:56:30,326 - mmdet - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offset.bias, layer3.4.conv2.conv_offset.weight, layer3.4.conv2.conv_offset.bias, layer3.5.conv2.conv_offset.weight, layer3.5.conv2.conv_offset.bias, layer3.6.conv2.conv_offset.

In [10]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...


Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7ff161452cb0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/tempfile.py", line 448, in __del__
    self.close()
  File "/opt/conda/lib/python3.7/tempfile.py", line 444, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmp284sq6rq/tmprb_eqqvm.py'


Done (t=3.56s)
creating index...
index created!


In [11]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 13:56:38,377 - mmdet - INFO - load checkpoint from /opt/ml/code/mmdetection_trash/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-b5f6da5e.pth
2021-05-19 13:56:38,377 - mmdet - INFO - Use load_from_local loader


Done (t=1.21s)
creating index...
index created!


2021-05-19 13:56:38,756 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.vfnet_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([11, 256, 3, 3]).
size mismatch for bbox_head.vfnet_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([11]).
2021-05-19 13:56:38,766 - mmdet - INFO - Start running, host: root@cf26aeee3585, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x
2021-05-19 13:56:38,767 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs
2021-05-19 13:58:20,027 - mmdet - INFO - Epoch [1][50/655]	lr: 1.882e-03, eta: 8:48:53, time: 2.025, data_time: 0.050, memory: 5266, loss_cls: 1.8300, loss_bbox: 0.2592, loss_bbox_rf: 0.3354, loss: 2.4246, grad_norm: 7.4205
2021-05-19 13:59:57,828 - mmdet - INFO - Epoch [1][100/655]	lr: 2.782e-03, eta: 8:38:12, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 7.5 task/s, elapsed: 88s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 8.1 task/s, elapsed: 81s, ETA:     0s

2021-05-19 23:10:07,373 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.60s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.82s).
Accumulating evaluation results...


2021-05-19 23:10:16,170 - mmdet - INFO - Epoch(val) [24][655]	bbox_mAP: 0.2920, bbox_mAP_50: 0.3910, bbox_mAP_75: 0.3060, bbox_mAP_s: 0.0490, bbox_mAP_m: 0.2570, bbox_mAP_l: 0.5220, bbox_mAP_copypaste: 0.292 0.391 0.306 0.049 0.257 0.522


DONE (t=1.26s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.292
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.391
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.306
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.049
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.257
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.522
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.460
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.460
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.460
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.141
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.426
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.749


In [12]:
!nvidia-smi

Wed May 19 23:10:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   73C    P0    70W / 250W |   6059MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
# push_image(cfg, checkpoint_path, wandb_name=wandb_name, img_num=10, mode="val", wandb_finish=True)